In [334]:
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import json
from shapely.geometry import Point

In [ ]:
# notebook setting

pd.options.display.max_columns = None

In [358]:
# methods

def read_shst_extract(path, suffix):
    shst_gdf = pd.DataFrame()
    
    shst_file = glob.glob(path + suffix)
    print("----------start reading shst extraction data-------------")
    for i in shst_file:
        print("reading shst extraction data : ", i)
        new = gpd.read_file(i)
        new['source'] = i
        shst_gdf = pd.concat([shst_gdf, new],
                             ignore_index = True,
                             sort = False)
    print("----------finished reading shst extraction data-------------")
    return shst_gdf

def extract_osm_link_from_shst_shape(x):
    """
    if len(x.get("metadata").get("osmMetadata").get("waySections")) > 1:
        link_df = pd.DataFrame()
        all_link_df = pd.DataFrame(x.get("metadata").get("osmMetadata").get("waySections"))
        '''
        link_df = pd.Series(data = {"nodeIds" : all_link_df.nodeIds.tolist(),
                               "wayId" : all_link_df.wayId.tolist(),
                               "roadClass" : all_link_df.roadClass.tolist(),
                               "oneWay" : all_link_df.oneWay.tolist(),
                               "name" : all_link_df.name.tolist()})
        link_df = pd.DataFrame(data = link_df)
        print(link_df)
        '''
        for c in all_link_df.columns.tolist():
            attr_list = all_link_df[c].tolist()
            final = []
            if c == "nodeIds":
                attr_list = [item for sublist in attr_list for item in sublist]
            link_df[c] = [attr_list] * 1
                
    else:    
        link_df = pd.DataFrame(x.get("metadata").get("osmMetadata").get("waySections"))
    """
    link_df = pd.DataFrame(x.get("metadata").get("osmMetadata").get("waySections"))
    link_df["geometryId"] = x.get("metadata").get("geometryId")
    
    shst_link_df_list.append(link_df)

"""
def extract_osm_link_from_shst_shape_pdconcat(x):
    
    link_df = pd.DataFrame(x.get("metadata").get("osmMetadata").get("waySections"))
    link_df["geometryId"] = x.get("metadata").get("geometryId")
    
    shst_link_df = pd.concat([shst_link_df, link_df], sort = False, ignore_index = True)
""" 
    
def osm_link_with_shst_info(link_df, shst_gdf):
    """
    get complete osm links with shst info
    1. two way shst to two osm links
    2. add u, v node
    
    Parameters
    --------------
    osm link from shst extraction
    
    return
    --------------
    complete osm link with shst info
    """
    osm_link_gdf = pd.merge(link_df, 
                            shst_gdf.drop(["roadClass", "metadata", "source"], axis = 1),
                           how = "left",
                           left_on = "geometryId",
                           right_on = "id")
    
    return osm_link_gdf
    
    
def add_two_way_osm(link_gdf, osmnx_link):
    """
    for osm with oneway = False, add the reverse direction to complete
    
    Parameters
    ------------
    osm link from shst extraction, plus shst info
    
    return
    ------------
    complete osm link
    """
    osm_link_gdf = link_gdf.copy()
    osm_link_gdf["wayId"] = osm_link_gdf["wayId"].astype(int)
    osm_link_gdf.drop("name", axis = 1, inplace = True)
    
    osmnx_link_gdf = osmnx_link.copy()
    
    """
    osmnx_link_gdf.rename(columns = {"u" : "u_for_osm_join",
                                     "v" : "v_for_osm_join"},
                         inplace = True)
    """
    osmnx_link_gdf.drop_duplicates(subset = ["osmid"], inplace = True)
    osmnx_link_gdf.drop(["length", "u", "v", "geometry"], axis = 1, inplace = True)
    
    print("shst extraction has geometry: ", osm_link_gdf.id.nunique())
    print("osm links from shst extraction: ", osm_link_gdf.shape[0])
    
    osm_link_gdf["u"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[0]))
    osm_link_gdf["v"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[-1]))
    
    print("---joining osm shst with osmnx data---")
    osm_link_gdf = pd.merge(osm_link_gdf,
                            osmnx_link_gdf,
                            left_on = ["wayId"],
                            right_on = ["osmid"],
                            how = "left")
    
    """
    #join on osmid, u, v
    osm_link_gdf["u_for_osm_join"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[0]))
    osm_link_gdf["v_for_osm_join"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[1]))
    
    """
    
    osm_link_gdf["oneWay"] = osm_link_gdf.apply(lambda x: True if True in [x.oneWay, x.oneway] else x.oneWay,
                                               axis = 1)
    
    reverse_osm_link_gdf = osm_link_gdf[(osm_link_gdf.oneWay == False) & 
                                        (osm_link_gdf.forwardReferenceId != osm_link_gdf.backReferenceId) & 
                                        (osm_link_gdf.u != osm_link_gdf.v)].copy()
    
    print("which includes two way links:", reverse_osm_link_gdf.shape[0])
    print("and they are geometrys: ", reverse_osm_link_gdf.id.nunique())
    
    reverse_osm_link_gdf.rename(columns = {"u" : "v",
                                          "v" : "u",
                                           #"u_for_osm_join" : "v_for_osm_join",
                                           #"v_for_osm_join" : "u_for_osm_join",
                                          "forwardReferenceId" : "backReferenceId",
                                          "backReferenceId" : "forwardReferenceId",
                                          "fromIntersectionId" : "toIntersectionId",
                                          "toIntersectionId" : "fromIntersectionId"},
                               inplace = True)
    
    reverse_osm_link_gdf["reverse_out"] = 1
    
    osm_link_gdf = pd.concat([osm_link_gdf, reverse_osm_link_gdf],
                            sort = False,
                            ignore_index = True)
    
    osm_link_gdf.rename(columns = {"forwardReferenceId" : "shstReferenceId",
                                 "geometryId" : "shstGeometryId"},
                      inplace = True)
    
    osm_link_gdf.drop("backReferenceId",
                     axis = 1,
                     inplace = True)
    """
    # join with osmnx
    print("---joining osm shst with osmnx data---")
    #col_before_join = osm_link_gdf.columns.tolist()
    osm_link_gdf = pd.merge(osm_link_gdf,
                            osmnx_link_gdf,
                            left_on = ["wayId"],#, "u_for_osm_join", "v_for_osm_join"],
                            right_on = ["osmid"],#, "u_for_osm_join", "v_for_osm_join"],
                           how = "left")
    """
    """
    succ_osm_link_gdf = osm_link_gdf[osm_link_gdf.osmid.notnull()].copy()
    print("-----number of matched osm------- :", succ_osm_link_gdf.shape[0])
    
    fail_osm_link_gdf = osm_link_gdf[osm_link_gdf.osmid.isnull()].copy()
    fail_osm_link_gdf = fail_osm_link_gdf[col_before_join].copy()
    print("-----number of un-matched osm-------:", fail_osm_link_gdf.shape[0])
    
    fail_osm_link_gdf["u_for_osm_join"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[-1]))
    fail_osm_link_gdf["v_for_osm_join"] = osm_link_gdf.nodeIds.apply(lambda x: int(x[-2]))
    
    fail_osm_link_gdf = pd.merge(fail_osm_link_gdf,
                            osmnx_link_gdf.drop("geometry", axis = 1),
                            left_on = ["wayId", "u_for_osm_join", "v_for_osm_join"],
                            right_on = ["osmid", "u_for_osm_join", "v_for_osm_join"],
                           how = "left")
    
    print("-----number of un-matched osm after rejoining-------:", fail_osm_link_gdf.shape[0])
    
    osm_link_gdf = pd.concat([succ_osm_link_gdf, fail_osm_link_gdf], ignore_index = True, sort = False)
    """
    
    
    print("after join, osm links from shst extraction: ", 
          len(osm_link_gdf), 
          " out of which there are ", 
          len(osm_link_gdf[osm_link_gdf.osmid.isnull()]), 
          " links that do not have osm info, due to shst extraction (default tile 181224) contains ", 
          osm_link_gdf[osm_link_gdf.osmid.isnull()].wayId.nunique(), 
          " osm ids that are not included in latest OSM extraction, e.g. private streets, closed streets.")
    print("after join, there are shst geometry # : ", osm_link_gdf.groupby(["shstReferenceId", "shstGeometryId"]).count().shape[0])
    
    return osm_link_gdf


def consolidate_osm_way_to_shst_link(osm_link):
    """
    if a shst link has more than one osm ways, aggregate info into one, e.g. series([1,2,3]) to cell value [1,2,3]
    
    Parameters
    ----------
    osm link with shst info
    
    return
    ----------
    shst link with osm info
    
    """
    osm_link_gdf = osm_link.copy()

    agg_dict = {"geometry" : lambda x: x.iloc[0],
                "u" : lambda x: x.iloc[0],
                "v" : lambda x: x.iloc[-1]}
    
    for c in ['link', 'nodeIds', 'oneWay', 'roadClass', 'roundabout', 'wayId', 'access', 'area', 'bridge',
              'est_width', 'highway', 'junction', 'key', 'landuse', 'lanes', 'maxspeed', 'name', 'oneway', 'ref', 'service', 
              'tunnel', 'width']:
        agg_dict.update({c : lambda x: list(x) if len(list(x)) > 1 else list(x)[0]})
    
    print("-----start aggregating osm segments to one shst link for forward links----------")
    forward_link_gdf = osm_link_gdf[osm_link_gdf.reverse_out.isnull()].copy()
    
    if len(forward_link_gdf) > 0:
        forward_link_gdf = forward_link_gdf.groupby(
                                        ["shstReferenceId", "id", "shstGeometryId", "fromIntersectionId", "toIntersectionId"]
                                        ).agg(agg_dict).reset_index()
        forward_link_gdf["forward"] = 1
    else:
        forward_link_gdf = None
    
    print("-----start aggregating osm segments to one shst link for backward links----------")
    
    backward_link_gdf = osm_link_gdf[osm_link_gdf.reverse_out==1].copy()
    
    if len(backward_link_gdf) > 0:
        agg_dict.update({"u" : lambda x: x.iloc[-1],
                     "v" : lambda x: x.iloc[0]})    

        backward_link_gdf = backward_link_gdf.groupby(
                                        ["shstReferenceId", "id", "shstGeometryId", "fromIntersectionId", "toIntersectionId"]
                                        ).agg(agg_dict).reset_index()
    else:
        backward_link_gdf = None
    
    shst_link_gdf = None
    
    if (forward_link_gdf is None):
        print("back")
        shst_link_gdf = backward_link_gdf
        
    if (backward_link_gdf is None):
        print("for")
        shst_link_gdf = forward_link_gdf
        
    if (forward_link_gdf is not None) and (backward_link_gdf is not None):
        print("all")
        shst_link_gdf = pd.concat([forward_link_gdf, backward_link_gdf],
                                  sort = False,
                                  ignore_index = True)
        
    shst_link_gdf = gpd.GeoDataFrame(shst_link_gdf,
                                    crs = {'init': 'epsg:4326'})
    
    return shst_link_gdf


def create_node_gdf(link_gdf):
    """
    create shst node gdf from shst geometry
    
    Paramters
    ---------
    link_gdf:  shst links with osm info
    
    return
    ---------
    shst nodes with osm info
    
    """
    print("-------start creating shst nodes--------")
    # geometry only matches for forward direction
    forward_link_gdf = link_gdf[link_gdf.forward == 1].copy()
    
    # create point geometry from shst linestring
    forward_link_gdf["u_point"] = forward_link_gdf.apply(lambda x: Point(list(x.geometry.coords)[0]), axis = 1)
    forward_link_gdf["v_point"] = forward_link_gdf.apply(lambda x: Point(list(x.geometry.coords)[-1]), axis = 1)
    
    # get from points
    point_gdf = forward_link_gdf[["u", "fromIntersectionId", "u_point"]].copy()
    
    point_gdf.rename(columns = {"u" : "osm_node_id",
                      "fromIntersectionId" : "shst_node_id",
                      "u_point" : "geometry"},
                    inplace = True)
    
    # append to points
    point_gdf = pd.concat([point_gdf, forward_link_gdf[["v", "toIntersectionId", "v_point"]].rename(columns = 
                     {"v" : "osm_node_id",
                      "toIntersectionId" : "shst_node_id",
                      "v_point" : "geometry"})],
                     sort = False,
                     ignore_index = True)
    
    # drop duplicates
    point_gdf.drop_duplicates(subset = ["osm_node_id", "shst_node_id"], inplace = True)
    
    point_gdf = gpd.GeoDataFrame(point_gdf,
                                 crs = {'init': 'epsg:4326'})
    
    return point_gdf



def link_df_to_geojson(df, properties):
    """
    Author: Geoff Boeing:
    https://geoffboeing.com/2015/10/exporting-python-data-geojson/
    """
    geojson = {"type":"FeatureCollection", "features":[]}
    for _, row in df.iterrows():
        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"LineString",
                               "coordinates":[]}}
        feature["geometry"]["coordinates"] = [[x, y] for (x,y) in list(row["geometry"].coords)]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return geojson


def point_df_to_geojson(df: pd.DataFrame, properties: list):
    """
    Author: Geoff Boeing:
    https://geoffboeing.com/2015/10/exporting-python-data-geojson/
    """
    
    geojson = {"type": "FeatureCollection", "features": []}
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {},
            "geometry": {"type": "Point", "coordinates": []},
        }
        feature["geometry"]["coordinates"] = [row["geometry"].x, row["geometry"].y]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return geojson

# run

In [2]:
# input osm data

print("-------reading osmnx data---------")
osmnx_link_gdf = gpd.read_file("../osmnx_extraction/link.geojson")
osmnx_node_gdf = gpd.read_file("../osmnx_extraction/node.geojson")
print("-------finished reading osmnx data---------")

In [4]:
# input shst extraction data

%%time
shst_link_gdf = read_shst_extract("../shst_node_js_extraction/", "*.out.geojson")

----------start reading shst extraction data-------------
reading shst extraction data :  ../shst_node_js_extraction\mtc_14.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_1.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_2.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_3.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_4.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_5.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_6.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_7.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_8.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_9.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_10.out.geojson
reading shst extraction data :  ../shst_node_js_extraction\mtc_11.out.geojson
reading shst ex

In [9]:
print("--------removing duplicated shst extraction data---------")
print("before removing duplicates, shst extraction has geometry # : ", shst_link_gdf.shape[0])

shst_link_non_dup_gdf = shst_link_gdf.drop_duplicates(
    subset = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId'])

print("after removing duplicates, shst extraction has geometry # : ", shst_link_non_dup_gdf.shape[0])

--------removing duplicated shst extraction data---------
before removing duplicates, shst extraction has geometry # :  1237008
after removing duplicates, shst extraction has geometry # :  908281


In [20]:
%%time
print("-------extracting single osm ways by every shst geometry----------")
shst_link_df_list = []

temp = shst_link_non_dup_gdf.apply(lambda x: extract_osm_link_from_shst_shape(x),
                            axis = 1)

osm_link_df = pd.concat(shst_link_df_list)

-------extracting single osm ways by every shst geometry----------
Wall time: 33min 15s


In [270]:
# note, the sharedstreets extraction using default tile osm/planet 181224
osm_link_gdf = osm_link_with_shst_info(osm_link_df,
                                      shst_link_non_dup_gdf)

osm_link_gdf = add_two_way_osm(osm_link_gdf,osmnx_link_gdf)

shst extraction has geometry:  908281
osm links from shst extraction:  974897
---joining osm shst with osmnx data---
which includes two way links: 837993
and they are geometrys:  793482
after join, osm links from shst extraction:  1812890  out of which there are  135953  links that do not have osm info, due to shst extraction (default tile 181224) contains  38386  osm ids that are not included in latest OSM extraction, e.g. private streets, closed streets.
after join, there are shst geometry # :  1701763


In [331]:
%%time

# aggregate osm data back to shst geometry based links

link_gdf = consolidate_osm_way_to_shst_link(osm_link_gdf)

-----start aggregating osm segments to one shst link for forward links----------
-----start aggregating osm segments to one shst link for backward links----------
all
Wall time: 14min 39s


In [355]:
%%time

# create node gdf

node_gdf = create_node_gdf(link_gdf)

-------start creating shst nodes--------
Wall time: 2min 26s


In [ ]:
link_prop = link_gdf.drop(["geometry", "forward"], axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(link_gdf, link_prop)

with open("../tests/link.geojson", "w") as f:
    json.dump(link_geojson, f)

In [ ]:
node_prop = node_gdf.drop("geometry", axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(node_gdf, node_prop)

with open("../tests/node.geojson", "w") as f:
    json.dump(node_geojson, f)

# test

In [154]:
# one-way segments does not have backeReferenceId

print(osm_link_df[osm_link_df.oneWay == True].geometryId.nunique())
print(shst_link_non_dup_gdf[shst_link_non_dup_gdf.backReferenceId == ""].shape[0])

106311
106311


In [152]:
# example for two-way segments that have only one directionality, thus no need to get the reverse direction for complete osm
osm_link_gdf[osm_link_gdf.id == "e24f210c1efe20bef41839ff1bc11f16"]

,link,nodeIds,oneWay,roadClass,roundabout,wayId,shstGeometryId,id,fromIntersectionId,toIntersectionId,...,landuse,lanes,maxspeed,name,oneway,osmid,ref,service,tunnel,width
21549,False,"[5250019116, 2829411762, 5250019116]",False,Other,False,7881681,e24f210c1efe20bef41839ff1bc11f16,e24f210c1efe20bef41839ff1bc11f16,acad3a72268359c4fe4267a4bbed1c52,acad3a72268359c4fe4267a4bbed1c52,...,nan,nan,nan,Old Moraga Ranch Trail,False,7881681.0,nan,nan,nan,nan


In [225]:
# the bollard at Berkeley
osm_link_gdf[osm_link_gdf.wayId == 24024252]

,link,nodeIds,oneWay,roadClass,roundabout,wayId,shstGeometryId,id,fromIntersectionId,toIntersectionId,...,landuse,lanes,maxspeed,name,oneway,osmid,ref,service,tunnel,width
93831,False,"[260541175, 4265656988, 260541174]",False,Other,False,24024252,ef53746f39d064e3ffe56f29523463a1,ef53746f39d064e3ffe56f29523463a1,5b772f58fa36634c522d08f32a1464ad,4cb4ad8f54ce81f14c5fe6aa58cdf60d,...,nan,nan,nan,nan,False,24024252.0,nan,nan,nan,nan
1053520,False,"[260541175, 4265656988, 260541174]",False,Other,False,24024252,ef53746f39d064e3ffe56f29523463a1,ef53746f39d064e3ffe56f29523463a1,4cb4ad8f54ce81f14c5fe6aa58cdf60d,5b772f58fa36634c522d08f32a1464ad,...,nan,nan,nan,nan,False,24024252.0,nan,nan,nan,nan


In [229]:
pd.crosstab(osm_link_gdf.roadClass, osm_link_gdf.highway)

highway,bridleway,closed:path,corridor,cycleway,footpath,footway,junction,living_street,motorway,motorway_link,...,secondary_link,service,steps,tertiary,tertiary_link,track,trunk,trunk_link,unclassified,unclassified_link
roadClass,,,,,,,,,,,,,,,,,,,,,
Motorway,0,0,0,0,0,0,0,0,5378,7700,...,0,42,0,1,0,0,84,50,5,0
Other,230,2,410,13048,78,265032,0,0,0,3,...,1,210805,4879,10,0,25515,0,0,24,0
Primary,0,0,0,1,0,0,0,0,5,6,...,31,54,0,237,0,0,417,79,2,0
Residential,0,0,0,7,0,22,2,423,0,1,...,32,2825,0,3363,32,221,0,6,964,0
Secondary,0,0,0,0,0,1,0,0,0,6,...,1368,13,0,1957,18,0,132,11,103,0
Service,0,0,0,76,0,50,0,10,0,4,...,15,346355,0,57,7,100,0,0,448,0
Tertiary,0,0,0,26,0,0,0,0,0,2,...,85,91,0,92684,547,0,0,10,357,0
Trunk,0,0,0,0,0,0,0,0,53,15,...,0,0,0,8,1,0,5004,1316,2,0
Unclassified,0,0,0,2,0,0,0,0,0,0,...,5,523,0,244,8,28,0,0,25229,2


In [219]:
osm_link_gdf[osm_link_gdf.id != osm_link_gdf.shstGeometryId]
osm_link_gdf[osm_link_gdf.oneway != osm_link_gdf.oneWay]
osm_link_gdf[(osm_link_gdf.oneway == False) & (osm_link_gdf.oneWay == True)]
osm_link_gdf[(osm_link_gdf.oneway == True) & (osm_link_gdf.oneWay == False)]
pd.crosstab(osm_link_gdf.oneWay, osm_link_gdf.oneway)
#osm_link_gdf[(osm_link_gdf.oneway == True) & (osm_link_gdf.oneWay == False)]

oneway,False,True
oneWay,,
False,1548844,0
True,735,127358
